In [20]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker and set random seed for reproducibility
fake = Faker()
random.seed(42)

# Generate data
num_records = 1000000
data = []

cities = [
    "London", "Paris", "Berlin", "Madrid", "Rome", "Vienna", "Bucharest", "Budapest", "Warsaw", "Barcelona",
    "Munich", "Milan", "Prague", "Sofia", "Brussels", "Hamburg", "Stockholm", "Belgrade", "Dublin", "Copenhagen",
    "Amsterdam", "Athens", "Helsinki", "Lisbon", "Oslo", "Zurich", "Edinburgh", "Bratislava", "Luxembourg", 
    "Ljubljana", "Riga", "Vilnius", "Tallinn", "Reykjavik", "Sarajevo", "Zagreb", "Skopje", "Podgorica", 
    "Tirana", "Chisinau", "Valletta", "Nicosia", "Andorra la Vella", "Monaco", "San Marino", "Vaduz"
]

for _ in range(num_records):
    record = {
        'id': fake.uuid4(),
        'name': fake.name(),
        'age': random.randint(18, 80),
        'city': random.choice(cities),
        'purchase_amount': round(random.uniform(10, 10000), 2),
        'purchase_date': fake.date_between(start_date='-1y', end_date='today')
    }
    data.append(record)

bronze_df = pd.DataFrame(data)

bronze_df['purchase_date'] = pd.to_datetime(bronze_df['purchase_date'])


# Bonze layer 
bronze_df.head()


,id,name,age,city,purchase_amount,purchase_date
0,c658a49a-413b-4757-8d02-3f89f818ed5e,Melanie Liu,58,Budapest,259.86,2024-03-13
1,99cb68f2-1f75-441a-a9f0-749a8afd3338,Kathryn Shannon,35,Hamburg,2239.88,2024-01-15
2,9124fbb9-9220-4947-b092-b9e55420a0c0,Kimberly Hernandez,65,Bucharest,6770.23,2024-05-11
3,bb3461de-aa58-4e7e-b15c-7e12890ea3dd,Lori Reilly,75,Sarajevo,878.52,2024-01-21
4,ae3ea75e-a5cc-49de-a119-4a9f33fb28eb,Angelica Reyes,45,Berlin,307.67,2023-10-13


In [21]:
bronze_df.to_csv('generated_data/medallion_1M.csv', index=False)

In [12]:
# Normalizing purchase_amount
bronze_df['normalized_purchase_amount'] = (bronze_df['purchase_amount'] - bronze_df['purchase_amount'].min()) / (bronze_df['purchase_amount'].max() - bronze_df['purchase_amount'].min())

# Extracting month and year from purchase_date
bronze_df['purchase_month'] = bronze_df['purchase_date'].dt.month
bronze_df['purchase_year'] = bronze_df['purchase_date'].dt.year

# Creating age_group column
bins = [17, 29, 39, 49, 59, 69, 79, 89]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
bronze_df['age_group'] = pd.cut(bronze_df['age'], bins=bins, labels=labels, right=True)

silver_df = bronze_df.copy()

silver_df.head()

,id,name,age,city,purchase_amount,purchase_date,normalized_purchase_amount,purchase_month,purchase_year,age_group
0,3f9c9796-2627-4f75-9326-c37042355e6c,James Smith,58,New York,259.86,2023-08-10,0.023361,8,2023,50-59
1,8d599c0a-c0b7-4a79-931f-f4d4aefd0d12,Bonnie Davis,35,Los Angeles,2239.88,2024-02-23,0.221915,2,2024,30-39
2,4f6f93de-162d-450c-ba6b-fa92a1beca16,Denise Flores,65,New York,6770.23,2024-02-19,0.676214,2,2024,60-69
3,7b3a2273-c5b4-4422-85df-7ce8ec89bf24,Stephanie Richardson,75,Phoenix,878.52,2023-11-09,0.085400,11,2023,70-79
4,446730de-f173-414a-905c-6ba2be64f403,Jessica Nelson,45,New York,307.67,2023-10-13,0.028155,10,2023,40-49


In [14]:
# Total purchase amount per city
total_purchase_per_city = silver_df.groupby('city')['purchase_amount'].sum().reset_index()

# Average purchase amount per age group
avg_purchase_per_age_group = silver_df.groupby('age_group')['purchase_amount'].mean().reset_index()

# Monthly total purchase amount
monthly_total_purchase = silver_df.groupby(['purchase_year', 'purchase_month'])['purchase_amount'].sum().reset_index()


total_purchase_per_city.head(), avg_purchase_per_age_group.head(), monthly_total_purchase.head()


(          city  purchase_amount
 0      Chicago       1055703.99
 1      Houston       1019678.81
 2  Los Angeles       1002767.30
 3     New York        900563.09
 4      Phoenix        969867.22,
   age_group  purchase_amount
 0     18-29      4790.771751
 1     30-39      5119.466606
 2     40-49      5069.838417
 3     50-59      4854.635174
 4     60-69      5060.045906,
    purchase_year  purchase_month  purchase_amount
 0           2023               6        194376.66
 1           2023               7        430231.38
 2           2023               8        445270.42
 3           2023               9        349029.30
 4           2023              10        342018.12)